use Selenium
2/22

爬取公視新聞網的新聞
>1. [政治](https://news.pts.org.tw/category/1)  x
>2. [全球](https://news.pts.org.tw/category/4)
>3. [生活](https://news.pts.org.tw/category/5)
>4. [地方](https://news.pts.org.tw/category/11)
>6. [兩岸](https://news.pts.org.tw/category/9) x
>7. [產經](https://news.pts.org.tw/category/10) x
>8. [社會](https://news.pts.org.tw/category/7)

Politics, National, International, Society, Finance, 



| Title     | Description  |
| ----------|:-------------:|
| Label     | Real = 1, Fake = 0
| Headline  | Headline of News            |
| Link      | url of News |
| Date      | Published Date      |  
| Source    | Source of the news. (One who can verify the claim of the news)| 
| Category  | Category of News            |
| Content   | Article or body of the news |

公視新聞網的網頁性質是，dcard(json)+報橘(post)



In [279]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [280]:
# 利用網址的編號為csv檔命名

cat = {
    1 : "politics",
    4 : "international",
    5 : "society",
    6 : "education",
    11 : "society1",
    9 : "international1",
    10 : "finance",
    7 : "society2",
    3 : "enviromental"
}


In [281]:
# 建立pandas表格
table = {
    "Headline" : [],
    "Link" : [],
    "Date" : [],
    "Source" : [],
    # "Category" : [],
    "Content" : []
}

In [282]:
# headers = {
#     "Cookie" : "_ga=GA1.3.1522906716.1622097110; _smt_uid=60bdea83.117c1ae2; _fbp=fb.2.1623059121706.1126395912; _gid=GA1.3.1044393895.1623893326; _gat_gtag_UA_4916790_28=1; XSRF-TOKEN=eyJpdiI6IjNTSEU1K3lONHFHNHZMSCtxK01JMGc9PSIsInZhbHVlIjoiekZtNkJUdnkxZVkvMmVzWTVOWm9ZNzJTeTNyd0hXb1g5eVRFdXFVY0F5ZWxTNFVqMEJVNmpFS0FwdURJRExDYmQ5YnhnM1dxRTFZSU9NUHZqK01nQTNRRm1LM29iWUpWazVpdGlacFFWTUFiMzJzQ3NxRHltM1pWa29seEM0cWciLCJtYWMiOiJiYzk0OGU3YmU1NTA1ZjY2NmM5NGIyYjcyMGNhYmRkZTJhODZkOTllNjY5YzU4MGQ5YmRiODk5MGI4NDYyYWViIn0%3D; laravel_session=eyJpdiI6IlNDdnNVb1JuZFB6L1lKc2NMT2xSZUE9PSIsInZhbHVlIjoiczhBS1BpUEdBM05YY25MbGRraVowdGFxaVU5K242d2NodUtTUnNSL0NLNzd5WElFZWRrK0dqTEhvTWx6R1JLVktDZnZDL1lZUDBwcmtkRzZzMmpqTU5ma3NKUDRnZExxT3diYWFJUzZydkV2OVY4UHdXNm10VW5sVVZDOFNkMEoiLCJtYWMiOiI0NGY5YzhmYTMwZmEzNzEwMGQyNmE3ODlkNjc2OGU4YTQwMjc1M2ZjMGEwMDU2MWM2MmM2ODNiOWE2NTVlNzlmIn0%3D; _gat_UA-4916790-28=1",
#     "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36"
# }

# i = 0
# form = {
#     "page" : (i + 1),
#     "cid" : 69
# }

In [283]:
import requests.packages.urllib3 # 解決InsecureRequestWarning
requests.packages.urllib3.disable_warnings()

In [284]:
# 建立取得source, content的自定義函數

def get_sc(link):
    response_ = requests.get(link, verify = False)
    html_ = BeautifulSoup(response_.text)

    source = html_.find("span", class_ = "article-reporter").text
    table["Source"].append(source)
    print(source)

    contents = html_.find("article", class_ = "post-article")
    # cname = contents.name
    # print(cname) # 顯示tag name
    # 處理內文是用div或p tag包的狀況

    if [x for x in ["article"]]:
        # 處理contents下沒有其他tag的問題
        # 因為contents只有一個，不需用for-in逐次抓出文字
        contents = html_.find("article", class_ = "post-article")
        content = contents.text.strip()

        # 直接利用div視為第1段...第n段，再使用for-in依序取得內文
        # 再將每個p以list格式合併儲存，而後再使用join使其裝在一起
        li = []
        li.append(content)
        joinli = ",".join(li)
        # print(joinli)
        table["Content"].append(joinli)
        print("article tag")
    elif [x for x in ["p", "div"]]: # for x in ["p", "div"]:
        li = []
        t = contents.find_all(x)
        for line in t:
            content = line.text.strip()
            li.append(content)
            joinli = ",".join(li)
            print(joinli)
        print("p/div")
        table["Content"].append(joinli)
    return source, joinli
# get_sc("https://news.pts.org.tw/article/530686") # div/p
# get_sc("https://news.pts.org.tw/article/530564") # article
# print(table)
# df = pd.DataFrame(table)
# df

In [285]:
# 取得各新聞網址、標題、發布時間

for i in range(80):
    i = i + 1
    print("page:", i)
    url = "https://news.pts.org.tw/category/3?page=" + str(i)
    
    response = requests.get(url, verify = False)
    response.encoding = "utf-8" # 解決中文亂碼問題
    html = BeautifulSoup(response.text)
    # print(html)

    # 特例: 因頭條新聞的html結構不同，個別爬取
    bn = html.find("div", class_ = "break-news-content").find("div", class_ = "break-news-container").find("div", class_ = "breakingnews")
    # print(bn)
    # breaknews_t = bn.select("h2 a") # select的功能是，可以指定h2標籤下的a標籤
    breaknews_t = bn.find("h2").find("a").text
    table["Headline"].append(breaknews_t)
    # print(breaknews_t)
    breaknews_l = bn.find("h2").find("a")["href"]
    table["Link"].append(breaknews_l)
    print(breaknews_l)
    # 利用自定義函數，取得頭條新聞的source, content
    get_sc(breaknews_l)

    breaknews_d = bn.find("div", class_ = "news-info").find("time").text
    table["Date"].append(breaknews_d)
    print("=" * 20)

    posts = html.find("ul", class_ = "list-unstyled news-list").find_all("li", class_ = "d-flex")
    # print(posts)
    for post in posts:
        title = post.find("h2")["title"]
        table["Headline"].append(title)
        print(title)
        link = post.find("h2").find("a")["href"]
        table["Link"].append(link)
        print(link)
        # 利用get_sc爬取個別新聞的內文、資料來源     
        get_sc(link)

        date = post.find("div", class_ = "news-info").find("time")["datetime"]
        table["Date"].append(date)

        print("=" * 20)




page: 1
https://news.pts.org.tw/article/533470
葉明蘭 / 整理報導
article tag
禽流感病毒不斷重組、變異 動保團體籲廢活禽交易
https://news.pts.org.tw/article/533464
黃怡菁 郭俊麟 / 台北報導
article tag
無薪假人數較上星期增加三成 新北市通報最多約3千人
https://news.pts.org.tw/article/533408
黃怡菁 郭俊麟 /新北市報導
article tag
核廢料無處處置 核二1號機今晚提前停機
https://news.pts.org.tw/article/533402
林靜梅 王興中 / 台北報導
article tag
機組員檢疫新制上路 海外接種疫苗如何證明？民航局仍研議中
https://news.pts.org.tw/article/533289
薛宜家 林志堅 / 綜合報導
article tag
台股早盤高點1萬7863點 賣壓湧現後翻黑
https://news.pts.org.tw/article/533282
郭品慈 / 綜合報導
article tag
寬鬆資金支撐 首季住宅價格指數新高
https://news.pts.org.tw/article/533281
吳雅瑜 陳立峰 / 台北報導
article tag
漁民釋放禁捕巨口鯊 花縣府頒獎表揚
https://news.pts.org.tw/article/533272
溫嘉楷 / 花蓮報導
article tag
空品惡化應變門檻降低 燃煤機組、鋼鐵、石化業納入強制降載
https://news.pts.org.tw/article/533201
林靜梅 吳嘉堡 / 台北報導
article tag
熱浪襲加拿大西部 49.5度高溫逾230人猝死
https://news.pts.org.tw/article/533194
王蕙文 / 編譯
article tag
page: 2
https://news.pts.org.tw/article/533101
洪蕙竹/ 綜合報導
article tag
太平島棘冠海星大爆發 造成99%珊瑚死亡
https://news.pts.org.tw/article/533092
林靜梅 吳嘉堡 / 台

In [286]:
df = pd.DataFrame(table)
df

,Headline,Link,Date,Source,Content
0,美股上漲帶動 台股早盤最高17795.88點,https://news.pts.org.tw/article/533470,\n 2021-07-...,葉明蘭 / 整理報導,美股道瓊工業指數上漲逾百點，標準普爾指數收高，日韓股同步走高，台股今天早盤跳空上漲，加權指數...
1,禽流感病毒不斷重組、變異 動保團體籲廢活禽交易,https://news.pts.org.tw/article/533464,2021-07-02 12:56:15,黃怡菁 郭俊麟 / 台北報導,環南市場爆發新冠群聚，動保團體提醒，只有一路之隔的家禽市場，則有「禽流感」這項人畜共通傳染病...
2,無薪假人數較上星期增加三成 新北市通報最多約3千人,https://news.pts.org.tw/article/533408,2021-07-01 19:51:11,黃怡菁 郭俊麟 /新北市報導,無薪假再擴大，今天公布的無薪通報企業有1305家、勞工1萬3626人，比上個星期增加三成。受...
3,核廢料無處處置 核二1號機今晚提前停機,https://news.pts.org.tw/article/533402,2021-07-01 19:51:11,林靜梅 王興中 / 台北報導,核二廠1號機的運轉執照尚未到期，但用過燃料無法退出爐心，今天晚上就要降載停機。環保團體認為，...
4,機組員檢疫新制上路 海外接種疫苗如何證明？民航局仍研議中,https://news.pts.org.tw/article/533289,2021-07-01 12:56:11,薛宜家 林志堅 / 綜合報導,七月起，我國實施新版機組員返台檢疫措施，未打疫苗或是接種第一劑未滿兩週，返國後要七天居家檢疫...
...,...,...,...,...,...
795,世界環境日前夕 環團籲速成立環境資源部,https://news.pts.org.tw/article/481089,2020-06-01 12:53:36,林靜梅 陳昌維 / 台北報導,北中南的環保團體，共同呼籲蔡政府，應讓環境資源部儘快成立。他們認為環境問題涉及氣候變遷調適、...
796,澎湖尖山電廠沙灘 3公尺領航鯨擱淺,https://news.pts.org.tw/article/480953,2020-05-30 19:20:08,黃子杰 陳立峰 / 報導,澎湖尖山發電廠後方沙灘，今天發現一隻身長310公分的領航鯨擱淺，澎湖縣府火速派員前往救援，金...
797,苗栗淺山區近百件光電廠規劃 將達百公頃,https://news.pts.org.tw/article/480905,2020-05-29 19:48:48,王龍韜 / 苗栗報導,再生能源是政府重要政策，停耕的農牧用地鼓勵轉型為太陽能光電場域。但保育人士發現，苗栗縣的淺山...
798,新竹湖口馬路被偷倒垃圾 逾半年未清完,https://news.pts.org.tw/article/480821,2020-05-29 12:52:29,張君豪 / 新竹報導,新竹湖口鄉一處大馬路上被人偷倒垃圾超過半年，到目前為止還沒清除完畢，現場有10噸多的垃圾，環...


In [287]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [288]:
# 利用網址的編號為csv檔命名
cate = url.split("/")[-1].split("?")[0]
cate = int(cate) # 將編號改為數字
cat[cate]

path = "drive/Shareddrives/Fake_news_detecter/datasets/PTS"
df.to_csv(path + "/" + cat[cate] + ".csv", encoding = "utf-8", index = False)